### <center>2018 Winter CS101.05</center>

# <center>基于数组的序列</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 课后作业-01

- 今日截止
- 习题讲解

## 今日议程

1. Python的序列类型
2. 底层数组结构
3. 动态数组和摊销
4. Python序列类型的效率
5. 多维数组

## Python的序列类型

Python内置了各种“序列”类，这些类底层都使用了类似数组的结构，它们都支持下标访问，如seq[k]：

- 列表(list)
- 元祖(tuple)
- 字符串(str)




本课程重点介绍序列的：

- 行为
- 实现细节
- 效率分析

## 底层数组结构

- 1byte = 8bit
- RAM(Ramdom Access Memory)
    - 认为存储器的任一单个字节被存储或检索的运行时间为O(1)
- 内存地址
- 广义上的数组结构：利用计算机储存器内的一块连续的存储空间，来存储一组具有相同类型的数据
    - 线性结构
    - 随机访问

![](./img/5-2.png)

数组根据下标随机访问的寻址公式：

> a[i]_address = base_address + i * data_type_size

### Python中的引用数组

Python内置的**list**和**tuple**类型底层使用内存地址来存储对象的引用

![](./img/5-3.png)


In [ ]:
primes = [2, 3, 5, 7, 11, 13, 17, 19]
temp = primes[3:6]

![](./img/5-4.png)

In [ ]:
temp[2] = 15
print(temp)
print(primes)

![](./img/5-5.png)

In [ ]:
counters = [0] * 8
counters

![](./img/5-6.png)

In [ ]:
counters[2] += 1
counters

![](./img/5-7.png)

In [ ]:
for i in range(len(counters)):
    print(id(counters[i]))

In [ ]:
extras = [23, 29, 31]
primes.extend(extras)
primes

![](img/5-8.png)

In [ ]:
id(extras[0]) == id(primes[8])

In [ ]:
extras[0] is primes[8]

### Python中的紧凑数组

- 不再像引用数组一样存储内存的地址，更高效利用内存空间
- 紧凑数组中的元素在内存里中是连续存放的
- array模块

In [ ]:
from array import array
primes = array('i', [2, 3, 5, 7, 11, 13, 17, 19])
primes

- array模块支持的类型

|参数|数据类型|字节数|
|---|---|---|
|'b'|signed char|1|
|'B'|unsigned char|1|
|'u'|Unicode char|2/4|
|'h'|signed short int|2|
|'H'|unsigned short int|2|
|'i'|signed int|2/4|
|'I'|unsigned int|2/4|
|'l'|signed long int|4|
|'L'|unsigned long int|4|
|'f'|float|4|
|'d'|double|8|

## 动态数组和均摊

Python的list类可以视为一种动态数组的实现

In [ ]:
import sys
data = []
for k in range(10):
    l = len(data)
    s = sys.getsizeof(data)
    print('List length: {0:3d}; Size in bytes: {1:4d}'.format(l, s))
    data.append(k)

### 实现一个动态数组 DynamicArray

当数组A的容量达到上限，需要插入新的元素时，扩展数组A的步骤如下：

1. 新分配一个更大的数组B；
2. 令B[i]=A[i] (i=0,1,2...,n-1)；
3. 令A=B, 在新分配的空间内添加元素；

![](img/5-9.png)

In [ ]:
import ctypes


class DynamicArray:
    """A simplified Python list which provides low-level array.
    """

    def __init__(self):
        """Create an empty array when initialied.
        """
        self._len = 0
        self._cap = 1
        self._A = self._make_array(self._cap)

    def _make_array(self, cap):
        """Return new array with capacity.
        """
        return (cap * ctypes.py_object)()

    def __len__(self):
        """Return the number of elements in array.
        """
        return self._len

    def __getitem__(self, k):
        """Return the element of array which indexed at k.
        """
        if not 0 <= k < self._len:
            raise IndexError("wrong index")
        return self._A[k]

    def append(self, val):
        """Add element to the end of array.
        """
        if self._len == self._cap:
            self._resize(2 * self._cap)
        self._A[self._len] = val
        self._len += 1

    def _resize(self, cap):
        B = self._make_array(cap)
        for i in range(self._len):
            B[i] = self._A[i]
        self._A = B
        self._cap = cap

In [ ]:
da = DynamicArray()
for k in range(10):
    l = len(da)
    s = da._cap
    print('DynamicArray length: {0:3d}; capacity: {1:4d}'.format(l, s))
    da.append(k)

### 动态数组的均摊分析

假设数组长度为n，插入操作根据数组位置的不同分为两种情况：

1. 如果数组的容量未达到上限，此时直接在空闲位置插入数据，时间复杂度是O(1)
2. 如果数组的容量已达到上限，会先对数组进行扩容操作，然后在执行插入操作，因为扩容时需要将原数组所有数据拷贝到新的内存空间中，时间复杂度是O(n)


![](img/5-10.png)

情况(1)和情况(2)中的每次操作的发生概率相同，都是$ \frac{1}{n+1} $，所以根据加权平均可算出：

$ \sum_{i=1}^{n-1} \frac{1}{n+1} + n \times \frac{1}{n+1} = \frac{2n-1}{n+1} $

动态数组插入操作的时间复杂度是O(1)

### 简单测试Python list的append效率

In [ ]:
from time import perf_counter as pc

def compute_average(n):
    data = []
    start = pc()
    for _ in range(n):
        data.append(None)
    elapsed = pc() - start
    return elapsed / n * 1000_000_000

for k in (10**exp for exp in range(2, 8)):
    avg_time = compute_average(k)
    print("Python list size:%10d, Average time of insertion: %.2f ns" % (k, avg_time))

![](img/5-11.png)

## Python序列类型的效率

### list/tuple

- tuple比list有更高的内存利用效率，因为tuple是不可变的，所以不需要管理动态的空间
- tuple有和list同样的不可变操作，时间复杂度也和list一样


|操作|示例|时间复杂度|注释|
|---|---|---|---|
| Index | d[k] | O(1) | |
| Length | len(d) | O(1)| |
| Count | d.count(val) | O(n)| 计算val出现的次数 |
| Get index | d.index(val) | O(k+1) | |
| Contain | val in d | O(k+1) | 顺序检索序列 |
| Compare | d1 == d2 | O(k+1) | 与 !=, <, <=, >, >= 类同 |
| Slice | d[j:k] | O(k-j+1) | |
| Concat | d1 + d2 | O($n1+n2$) | |
| Multiply | C * d | O(cn) | |
| Extreme value | max(d) / min(d) | O(n) | 检索所有元素 |
| Interation | for v in d | O(n) | |

- list可变操作的时间复杂度

|操作|示例|时间复杂度|注释|
|---|---|---|---|
| Store | d[k] = 0 | O(1) | |
| Append | d.append(val) | O(1) | 均摊时间复杂度 |
| Insert | d.insert(k, val) | O(n-k+1) | 均摊时间复杂度 |
| Pop | d.pop() | O(1) | 同l.pop(-1)，从list末尾pop，均摊时间复杂度 |
| Pop kth | d.pop(k) | O(n-k) | 均摊时间复杂度 |
| Delete | del d[k] | O(n-k) | 均摊时间复杂度 |
| Remove | d.remove(val) | O(n) | 均摊时间复杂度 |
| Extend | d1.extend(d2) / d1 += d2 | O($n_2$) | 均摊时间复杂度 |
| Reverse | d.reverse() | O(n) | |
| Sort | d.sort() | O(nlogn) | |
| Copy | d.copy() | O(n) | 类似d[:] |
| Clear | d.clear() | O(1) | 类似l = [] |

### 简单测试Python list的insert效率

In [ ]:
from time import perf_counter as pc

def clock(func):
    def decorate(*args):
        start = pc()
        result = func(*args)
        elapsed = pc() - start
        return elapsed / result * 1000_000_000
    return decorate

@clock
def compute_insert_average(n, insert_pos=0.0):
    data = []
    for i in range(n):
        data.insert(int(n*insert_pos), None)
    return len(data)

for k in (10**exp for exp in range(2, 6)):
    print('-'*80)
    print("Python list size:%10d, Average time of insertion at head: %.2f ns" % (k, compute_insert_average(k, 0.0)))
    print("Python list size:%10d, Average time of insertion at mid: %.2f ns" % (k, compute_insert_average(k, 0.5)))
    print("Python list size:%10d, Average time of insertion at tail: %.2f ns" % (k, compute_insert_average(k, 1.0)))

#### DynamicArray实现insert方法

比如 da.insert(k, val):

1. 先检查DynamicArray的空间是否达到上限，如果达到就扩容2倍
2. da中在k索引之后的所有元素都向后依次移动一位
3. 把val值存储在da的k位上，然后da的长度增加1

![](img/5-12.png)

```python
def insert(self, k, val):
    """Insert value at index k and shifting subsequent values rightward.
    """
    if self._len == self._cap:
        self._resize(2 * self._cap)
    # shifting the sub array to right    
    for i in range(self._len, k, -1):
        self._A[i] = self._A[i-1]
    self._A[k] = val
    self._len += 1
```

#### DynamicArray实现remove方法

比如 da.remove(val):

1. 依次检索da中的元素是否等于val，一旦找到相同元素：
    1. 将此元素后的所有元素依次向前移动一位（最后一个元素置为None）
    2. 将da的长度减少1
2. 否则返回 ValueError

![](img/5-13.png)

```python
def remove(self, val):
    """Remove first occurrence of value, raise Value Error if not match.
    """
    for k in range(self._len):
        if self._A[k] == val:
            for i in range(k, self._len - 1):
                self._A[i] = self._A[i + 1]
            self._A[self._len - 1] = None
            self._len -= 1
            return
    raise ValueError('value not found')
```

### str

- 字符串是一种非常特殊的immutable序列，现实中的执行效率往往比想象中的高
- 有许多优化的算法来提高字符串的执行效率，比如匹配算法(CS101.13再详细介绍)
- Python的解释器对字符串也有一定的优化
```python
for c in long_long_string:
    if c.isalpha():
        letters += c
```
- 通过更Pythonic的方式来优化
```python
letters = ''.join([c for c in long_long_string if c.isalpha()])
letters = ''.join(c for c in long_long_string if c.isalpha())
```

In [ ]:
import random, string

long_long_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10000))


In [ ]:
%%timeit
letters = ''
for c in long_long_string:
    if c.isalpha():
        letters += c

In [ ]:
%timeit ''.join([c for c in long_long_string if c.isalpha()])
%timeit ''.join(c for c in long_long_string if c.isalpha())

## 多维数据集

- 能否可以像创建一维数组一样创建多维数组？

In [ ]:
array_2d = [ [0] * 3 ] * 4
array_2d

In [ ]:
array_2d[2][0] = 1
array_2d

![](img/5-14.png)

- 正确的写法

In [ ]:
array_2d = [ [0] * 3 for i in range(4) ]
array_2d[2][0] = 1
array_2d

- 更推荐使用Numpy/Pandas来创建多维数组（ML101中介绍）

# Any Questions?

## 课后作业 Assignment-03

1) 课上我们已经实现了一个简单的动态数组DynamicArray，在此基础上我们需要对其进行重构新增一些新的Feature：

a) 重构扩容的相关方法，使Dynamic在扩容时能更具不同的条件来选择扩大的容量。如果总容量小于1000时，扩容仍然采用达到上限是扩大2倍，但如果总容量超过了1000，扩容的策略更变为达到上限后扩大为原有容量的1.25倍，即每次扩容只增加25%的空间。

b) 新增一个pop()方法，此方法可以让DynamicArray删除并返回最后一个元素，并且为DynamicArray新增一个缩容的feature，即每当数组中的元素个数小于总容量的四分之一时，DynamicArray会自动缩容到原来容量的一半。

P.S. 原DynamicArray的代码托管在[Github](https://github.com/shaqsnake/Data-Structures-and-Algorithms-in-Python/blob/master/src/ch05/dynamic_array.py)和[CodeClub](http://code.huawei.com/t00361654/CS101-DataStructures-and-Algorithms/blob/master/src/ch05/dynamic_array.py)上，可以自行参考，但建议看懂后能自己先重新实现一遍再开始做本题。
